In [76]:
from langgraph.graph import StateGraph, START , END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from pydantic import BaseModel , Field
import operator


In [77]:
load_dotenv()

True

In [78]:
import os
api_key = os.environ.get("PERPLEXITY_API_KEY")


In [79]:
model = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.perplexity.ai",
    model="sonar-pro"  # or another supported Perplexity model
)

In [80]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description = 'detailed feedback for essay')
    score:int = Field(description='Score out of 10', getattr = 0 , le= 10)

In [81]:
structured_model = model.with_structured_output(EvaluationSchema)

In [82]:
essay = """The Role of India in Artificial Intelligence: Shaping the Present and Future

In the past decade, India has emerged as a vital, dynamic force in the global landscape of artificial intelligence (AI). Once seen as a technology consumer, India is now an innovator, ecosystem builder, and advocate for democratized, responsible AI. This remarkable trajectory stems from a unique confluence of government vision, entrepreneurial energy, world-class talent, and the country’s emphasis on inclusivity and public good.

1. Vision and Strategy: Democratizing AI for All
Unlike approaches centered solely on commerce or military use, India’s strategy roots itself in “AI for All”—seeking to ensure access, opportunity, and benefit across all strata of society. The government’s IndiaAI Mission (launched in 2024 with over ₹10,300 crore committed over five years) is constructing one of the world’s strongest foundations for AI, from massive GPU compute infrastructure to high-quality, anonymized datasets. Programs like Centres of Excellence in AI, the IndiaAI Dataset Platform (AIKosh), and public-good initiatives reinforce this approach, making advanced AI hardware and data accessible to startups, researchers, and students, not just tech giants or a privileged few.

India’s Digital Public Infrastructure serves as a global model by combining public funding and private innovation: foundational digital platforms like Aadhaar, UPI, and DigiLocker are now being “supercharged” with AI, inspiring adoption around the world.

2. Fostering Innovation and Indigenous Foundations
India’s push to develop homegrown foundational AI models is gaining momentum. The IndiaAI Innovation Initiative invites startups, labs, and academic researchers to build Large Language Models (LLMs) and domain-specific solutions tailored to India’s 23 major languages and distinctive societal needs. This not only addresses global challenges of linguistic diversity and cultural nuance in AI, but also fosters strategic autonomy—reducing reliance on foreign AI models.

More than 500 proposals for foundational models have been submitted by Indian startups and researchers in 2025 alone—an extraordinary outpouring of grassroots innovation.

3. Talent Engine and Global Influence
India boasts the highest AI skill penetration in the world, and a talent pipeline growing at breakneck speed. With over 420,000 AI professionals and widespread upskilling programs, India is recognized as a global AI talent hub. The affordable, abundant talent pool enables Indian startups not only to serve the domestic market but to create AI products for the world—making India the second-largest contributor to GitHub AI projects.

4. Economic and Societal Impact
By 2025, India’s AI sector is expected to contribute $500 billion to GDP, with industry bodies and global consultancies predicting millions of new, high-value jobs in the coming decade. AI is already transforming sectors crucial to India’s development: smart agriculture, affordable healthcare, inclusive education, urban planning, and secure financial services. As a “third way” between East and West, India’s AI model balances growth ambition with an ethos of ethical and inclusive AI, and carbon-neutral digital progress.

5. Global Engagement and Responsible AI
India is deeply engaged in shaping international AI norms. As co-chair of the Global Partnership on AI and through forums like the AI Governance Alliance, India champions “ethics by design,” data privacy safeguards, and AI for social good. Initiatives like RAISE (Responsible AI for Social Empowerment) exemplify this stance.

6. Challenges and the Road Ahead
India’s massive scale presents both opportunity and challenge. While there is a surge in innovation, addressing ethical, privacy, and security concerns, as well as bridging the digital skills gap, remain ongoing priorities. The government continues to invest in upskilling; legal frameworks are being adapted to keep pace with technology.

Conclusion
India’s role in artificial intelligence is no longer peripheral—it is central, visionary, and deeply human-centric. By combining indigenous innovation with global collaboration and ethical leadership, India is not only transforming its economy and society but also offering an inclusive, responsible model for the world as AI reshapes humanity’s future"""

In [83]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
structured_model.invoke(prompt).feedback

"The essay demonstrates a high level of language quality overall, with clear, sophisticated, and varied sentence structures. The vocabulary is advanced yet accessible, and the tone matches the serious, analytical nature of the topic. Transitions between paragraphs are smooth, with clear thematic progression. Each section maintains focus and coherence, and technical terms are explained within context, aiding understanding. There are minimal grammatical errors, if any. The introduction effectively sets the scene, while the conclusion provides a strong summative statement without unnecessary repetition. Some sentences, especially those in statistics-heavy sections, could be streamlined for conciseness, and a few phrases (like 'grassroots innovation') might benefit from precise quantification, but these are minor issues. Occasional use of complex, multi-clause sentences could impact readability for some audiences, but overall, the piece maintains clarity and flow. The essay excels at combi

In [84]:
class UPSCState(TypedDict):
    essay : str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback: str
    overall_feedback:str
    individual_scores:Annotated[list[int], operator.add]
    average_score: float

NameError: name 'Annotated' is not defined